In [2]:
import numpy as np
import pandas as pd
import math

### Problem 1

In [ ]:

def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def call_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European call option at time T."""
    return np.exp(-r * T) * np.maximum(Si - K, 0)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return np.exp(-r * T) * np.maximum(K - Si, 0)

def call_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * np.exp(-r * T) * Si / S0

def put_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * np.exp(-r * T) * Si / S0

def call_gamma_estimate(Si, S0, K, T, r, sigma, zi):
    """Calculate the gamma estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return  np.exp(-r * T) * indicator * np.exp(-q * T) * zi / (S0 * sigma * np.sqrt(T))

def put_gamma_estimate(Si, S0, K, T, r, sigma, zi):
    """Calculate the gamma estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return   np.exp(-r * T) * indicator * np.exp(-q * T) * zi / (S0 * sigma * np.sqrt(T))

def call_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * Si * np.exp(-r * T) * (-sigma * T + np.sqrt(T) * zi)

def put_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * Si * np.exp(-r * T) * (-sigma * T + np.sqrt(T) * zi)

def call_theta_estimate(Si, S0, K, T, r, q, sigma, zi):
    """Calculate the theta estimate of the European call option at time T"""
    
def put_theta_estimate(Si, S0, K, T, r, q, sigma, zi):
    """Calculate the theta estimate of the European put option at time T"""
    
def call_rho_estimate(Si, K, T, r, sigma, zi):
    """Calculate the rho estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * np.exp(-r * T) * K * T

def put_rho_estimate(Si, K, T, r, sigma, zi):
    """Calculate the rho estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * np.exp(-r * T) * K * T

def linear_congruential_generator(N, seed = 1):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = seed

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui

    return samples, xi


def marsaglia_bray(seed=1):
    X = 2  # set an initial value so that the while loop is entered
    i = 0
    while X > 1:
        U, seed = linear_congruential_generator(2, seed)
        u1, u2 = U[0],U[1]
        u1 = 2 * u1 - 1
        u2 = 2 * u2 - 1
        X = u1**2 + u2**2
        
    Y = math.sqrt(-2 * math.log(X) / X)
    Z1 = u1 * Y
    Z2 = u2 * Y
    
    return Z1, Z2, seed

def marsaglia_bray_N(N):
    ## Generate an empty array for the generated values.
    results = np.empty(N, dtype=float)
    index = 0
    seed = 1
    while index < N:
        Z1, Z2, seed = marsaglia_bray(seed)
        
        ## Add the generated values to the results array.
        results[index] = Z1
        index += 1
        
        ## If there's still space, add the second generated value.
        if index < N:
            results[index] = Z2
            index += 1

    return results

def MC_call(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    dt = 0.001
    Si_array = simulated_spot(S0, T, r, q, sigma, z)
    Si_array_dt = simulated_spot(S0, T+dt, r, q, sigma, z)
    Si_array_gamma_2 = simulated_spot(S0 + 0.01, T, r, q, sigma, z)
    Si_array_gamma_1 = simulated_spot(S0 + 0.005, T, r, q, sigma, z)

    ## Vectorized calculation for call payoff, delta, and vega.
    C_hat_array = call_payoff(Si_array, K, T, r)
    C_hat_array_dt = call_payoff(Si_array_dt, K, T+dt, r)
    C_hat_array_gamma_2 = call_payoff(Si_array_gamma_2, K, T, r)
    C_hat_array_gamma_1 = call_payoff(Si_array_gamma_1, K, T, r)

    delta_C_hat_array = call_delta_estimate(Si_array, S0, K, T, r)
    gamma_C_hat_array = (C_hat_array_gamma_2 + C_hat_array - 2 * C_hat_array_gamma_1) / 0.005 ** 2
    vega_C_hat_array = call_vega_estimate(Si_array, K, T, r, sigma, z)
    theta_C_hat_array = ( C_hat_array - C_hat_array_dt) / dt
    rho_C_hat_array = call_rho_estimate(Si_array, K, T, r, sigma, z)

    ## Aggregation.
    N = len(z)
    C_hat = np.mean(C_hat_array)
    delta_C_hat = np.mean(delta_C_hat_array)
    gamma_C_hat = np.mean(gamma_C_hat_array)
    vega_C_hat = np.mean(vega_C_hat_array)
    theta_C_hat = np.mean(theta_C_hat_array)
    rho_C_hat = np.mean(rho_C_hat_array)

    return C_hat, delta_C_hat, gamma_C_hat, vega_C_hat, theta_C_hat, rho_C_hat
def MC_put(S0, K, T, r, q, sigma, z):
    ## Vectorized calculation for all simulated spot prices.
    dt = 0.001
    Si_array = simulated_spot(S0, T, r, q, sigma, z)
    Si_array_dt = simulated_spot(S0, T+dt, r, q, sigma, z)
    Si_array_gamma_2 = simulated_spot(S0 + 0.01, T, r, q, sigma, z)
    Si_array_gamma_1 = simulated_spot(S0 + 0.005, T, r, q, sigma, z)
    # Vectorized calculation for put payoff, delta, and vega.

    P_hat_array = put_payoff(Si_array, K, T, r)
    P_hat_array_dt = put_payoff(Si_array_dt, K, T+dt, r)
    P_hat_array_gamma_2 = put_payoff(Si_array_gamma_2, K, T, r)
    P_hat_array_gamma_1 = put_payoff(Si_array_gamma_1, K, T, r)
    
    delta_P_hat_array = put_delta_estimate(Si_array, S0, K, T, r)
    gamma_P_hat_array = (P_hat_array_gamma_2 + P_hat_array - 2 * P_hat_array_gamma_1) / 0.005 ** 2
    vega_P_hat_array = put_vega_estimate(Si_array, K, T, r, sigma, z)
    theta_P_hat_array = ( P_hat_array - P_hat_array_dt) / dt
    rho_P_hat_array = put_rho_estimate(Si_array, K, T, r, sigma, z)

    ## Aggregation.
    N = len(z)
    P_hat = np.mean(P_hat_array)
    delta_P_hat = np.mean(delta_P_hat_array)
    gamma_P_hat = np.mean(gamma_P_hat_array)
    vega_P_hat = np.mean(vega_P_hat_array)
    theta_P_hat = np.mean(theta_P_hat_array)
    rho_P_hat = np.mean(rho_P_hat_array)

    return P_hat, delta_P_hat, gamma_P_hat, vega_P_hat, theta_P_hat, rho_P_hat

## Table values for #3.

## Given values.
S0 = 49
K = 52
sigma = 0.27
q = 0.02
r = 0.045
T = 7 / 12

## Initialize variables. 
N = 10_000
call_lst = []
for k in range(5,10):
    z = marsaglia_bray_N(N*(2**k))
    call_lst.append((MC_call(S0, K, T, r, q, sigma, z)))

put_lst = []
for k in range(5,10):
    z = marsaglia_bray_N(N*(2**k))
    put_lst.append((MC_put(S0, K, T, r, q, sigma, z)))

CALL = pd.DataFrame(call_lst, columns=["C_hat", "delta_C_hat", "gamma_C_hat", "vega_C_hat", "theta_C_hat", "rho_C_hat"], index = ['N = 320,000', 'N = 640,000', 'N = 1,280,000', 'N = 2,560,000', 'N = 5,120,000'])
PUT = pd.DataFrame(put_lst, columns=["P_hat", "delta_P_hat", "gamma_P_hat", "vega_P_hat", "theta_P_hat", "rho_P_hat"], index = ['N = 320,000', 'N = 640,000', 'N = 1,280,000', 'N = 2,560,000', 'N = 5,120,000'])

In [ ]:
CALL

In [ ]:
PUT

### Problem 2

In [2]:
np.random.seed(42)
# Given data
S = 62.0  # Current spot price
K = 66.0  # Strike price
T = 10 / 12  # Time to expiration (10 months)
r = 0.045  # Risk-free interest rate
dividend_yield = 0.015  # Continuous dividends
market_price = 6.36  # Market price of the option

# Number of simulations
num_simulations = 1000000

# Monte Carlo simulation function
def monte_carlo_simulation(S, K, T, r, sigma, num_simulations):
    z = np.random.standard_normal(num_simulations)
    ST = S * np.exp((r - dividend_yield - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    option_payoffs = np.maximum(K - ST, 0)
    option_price = np.exp(-r * T) * np.mean(option_payoffs)
    return option_price

def calculate_implied_volatility(S, K, T, r, market_price, tolerance=1e-6):
    # Initial guess for volatility (e.g., 0.2 or 20% as a decimal)
    volatility_low = 0.1
    volatility_high = 0.3
    # Binary search loop
    while (volatility_high - volatility_low) > tolerance:
        volatility_mid = (volatility_low + volatility_high) / 2.0
        option_price = monte_carlo_simulation(S, K, T, r, volatility_mid, num_simulations)
        # Update the bounds for the binary search
        if option_price < market_price:
            volatility_low = volatility_mid
        else:
            volatility_high = volatility_mid
    return volatility_mid

# Print the implied volatility with two decimal digits
implied_vol = calculate_implied_volatility(S, K, T, r, market_price) * 100
print(f"Implied Volatility: {implied_vol}%")

Implied Volatility: 22.503585815429688%


### Problem 3

In [3]:
def cal_delta(stock, option):
    return (option[1][0] - option[1][2]) / (stock[1][0] - stock[1][2])

def cal_gamma(stock, option):
    return (((option[2][0] - option[2][2]) / (stock[2][0] - stock[2][2])) - ((option[2][2] - option[2][4]) / (stock[2][2] - stock[2][4]))) / ((stock[2][0] - stock[2][4]) / 2)

def cal_theta(option, dt):
    return (option[1][1] - option[0][0]) / dt

def cal_vega(dv, dsigma):
    return (dv[0] - dv[1]) / dsigma

def cal_rho(dv, dr):
    return (dv[0] - dv[1]) / dr

def american_trinomial(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree for the option values
    call_option_tree = np.zeros((N + 1, 2*N + 1))
    put_option_tree = np.zeros((N + 1, 2*N + 1))
    stock_price_tree = np.zeros((N + 1, 2*N + 1))

    for j in range(2*N + 1):
        stock_price_tree[N][j] = S * (u ** (N - j))
        call_option_tree[N][j] = max(0, S * (u ** (N - j)) - K)
        put_option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            call_intrinsic = S * (u ** (i - j)) - K
            put_intrinsic = K - S * (u ** (i - j))
            stock_price_tree[i][j] = S * (u ** (i - j))
            call_option_tree[i][j] = max(call_intrinsic, np.exp(-r * dt) * (pu * call_option_tree[i + 1][j] + pm * call_option_tree[i + 1][j + 1] + pd * call_option_tree[i+1][j+2]))
            put_option_tree[i][j] = max(put_intrinsic, np.exp(-r * dt) * (pu * put_option_tree[i + 1][j] + pm * put_option_tree[i + 1][j + 1] + pd * put_option_tree[i+1][j+2]))

    call_value, put_value = call_option_tree[0][0], put_option_tree[0][0]
    call_delta, put_delta = cal_delta(stock_price_tree, call_option_tree), cal_delta(stock_price_tree, put_option_tree)
    call_gamma, put_gamma = cal_gamma(stock_price_tree, call_option_tree), cal_gamma(stock_price_tree, put_option_tree)
    call_theta, put_theta =  cal_theta(call_option_tree, dt), cal_theta(put_option_tree, dt)

    return [call_value, call_delta, call_gamma, call_theta], [put_value, put_delta, put_gamma, put_theta]

In [4]:
S = 33.0  # Current spot price
K = 32.0   # Strike price
sigma = 0.24  # Volatility
r = 0.045  # Risk-free rate
T = 10 / 12  # Time to expiration in years
q = 0.02  # Continuous dividends

dsigma = 1e-5 # to calculate vega
dr = 1e-5 # to calculate rho

time_steps_values = [10, 20, 40, 80, 160, 320, 640, 1280]

call_df = pd.DataFrame(columns=time_steps_values)
put_df = pd.DataFrame(columns=time_steps_values)

for N in time_steps_values:
    call, put = american_trinomial(S, K, T, r, q, sigma, N)
    call_sigma, put_sigma = american_trinomial(S, K, T, r, q, sigma+dsigma, N)
    call_r, put_r = american_trinomial(S, K, T, r+dr, q, sigma, N)
    call_vega, put_vega = cal_vega([call_sigma[0], call[0]], dsigma), cal_vega([put_sigma[0], put[0]], dsigma)
    call_rho, put_rho = cal_rho([call_r[0], call[0]], dr), cal_rho([put_r[0], put[0]], dr)

    call.extend([call_vega, call_rho])
    put.extend([put_vega, put_rho])
    call_df[N] = call
    put_df[N] = put

call_df = call_df.applymap(lambda x: round(x, 4))
put_df = put_df.applymap(lambda x: round(x, 4))

# in the order: value, delta, gamma, theta, vega, rho
call_df.to_csv('./call_value.csv')
put_df.to_csv('./put_value.csv')

### Problem 4

In [5]:
def american_trinomial_put(S, K, T, r, q, sigma, N):
    dt = T / N
    u = np.exp(sigma * np.sqrt(3*dt))
    d = 1 / u
    pu = 1/6 + (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))
    pm = 2/3
    pd = 1/6 - (r-q-sigma**2*0.5) * np.sqrt(dt/(12*sigma**2))

    # Build the binomial tree for the option values
    put_option_tree = np.zeros((N + 1, 2*N + 1))

    for j in range(2*N + 1):
        put_option_tree[N][j] = max(0, K - S * (u ** (N - j)))

    # Backward induction
    for i in range(N - 1, -1, -1):
        for j in range(2*i+1):
            put_intrinsic = K - S * (u ** (i - j))
            put_option_tree[i][j] = max(put_intrinsic, np.exp(-r * dt) * (pu * put_option_tree[i + 1][j] + pm * put_option_tree[i + 1][j + 1] + pd * put_option_tree[i+1][j+2]))

    return put_option_tree[0][0]

In [ ]:
# Given option information
S = 56
K = 60
T = 8/12
r = 0.045
q = 0.015
option_price = 6.95
tol = 1e-4
sigma = 0.3

N = 10
prev = 0
while True:
    curr= american_trinomial_put(S, K, T, r, q, sigma, N)
    if abs(curr - prev) < tol: break
    prev = curr
    N *= 2

# final iteration N
N_fixed = N

print('N_fixed: ', N_fixed)

In [8]:
def A(y):
    return (np.exp((1 - 2 / np.pi) * y) - np.exp(-(1 - 2 / np.pi) * y))**2

def B(y, R):
    return 4 * (np.exp(2 * y / np.pi) + np.exp(-2 * y / np.pi)) - 2 * np.exp(-y) * \
           (np.exp((1 - 2 / np.pi) * y) + np.exp(-(1 - 2 / np.pi) * y)) * (np.exp(2 * y) + 1 - R**2)

def C(y, R):
    return np.exp(-2 * y) * (R**2 - (np.exp(y) - 1)**2) * ((np.exp(y) + 1)**2 - R**2)

def funcA(x):
    if x >= 0:
        return 0.5 + 0.5 * np.sqrt(1 - np.exp(-2 * x**2 / np.pi))
    else:
        return 0.5 - 0.5 * np.sqrt(1 - np.exp(-2 * x**2 / np.pi))

def sr_appr(S, K, r, T, q, P):
    e = K * np.exp(-r * T) / (S * np.exp(-q * T))
    F = K / e
    y = np.log(F / K)
    alpha_put = P / (K * np.exp(-r * T))
    R = 2 * alpha_put + np.exp(y) - 1
    
    beta = 2 * C(y, R) / (B(y, R) + np.sqrt(B(y, R)**2 + 4 * A(y) * C(y, R)))
    gamma = -np.pi * np.log(beta) / 2

    if y >= 0:
        P0 = K * np.exp(-r * T) * (0.5 - np.exp(y) * funcA(-np.sqrt(2 * y)))
        if P <= P0:
            return (np.sqrt(gamma + y) - np.sqrt(gamma - y)) / np.sqrt(T)
        else:
            return (np.sqrt(gamma + y) + np.sqrt(gamma - y)) / np.sqrt(T)
    else:
        P0 = K * np.exp(-r * T) * (funcA(np.sqrt(-2 * y)) - np.exp(y) / 2)
        if P <= P0:
            return (-np.sqrt(gamma + y) + np.sqrt(gamma - y)) / np.sqrt(T)
        else:
            return (np.sqrt(gamma + y) + np.sqrt(gamma - y)) / np.sqrt(T)

sigma_initial = sr_appr(S, K, r, T, q, option_price)
sigma_plus = sigma_initial + 0.02
sigma_minus = sigma_initial - 0.02

output = pd.DataFrame()
result = {}
iteration = 0
N = 5120
while abs(sigma_plus - sigma_minus) > tol:
    iteration += 1
    curr = american_trinomial_put(S, K, T, r, q, sigma_plus, N)
    print(iteration, sigma_plus, curr)
    result[iteration] = (sigma_plus, curr)
    sigma = sigma_plus - (sigma_plus - sigma_minus) * (curr - option_price) / (curr - prev)
    sigma_plus, sigma_minus = sigma, sigma_plus
    prev = curr

data_df = pd.DataFrame(result)
data_df.to_csv('./prob4.csv')
print('implied vol: ', sigma)
print('option price: ', curr)

1 0.3145876563907092 7.491009007974748
2 0.2745267775383741 6.7810505694007075
3 0.2840600999184889 6.949343041734153
implied vol:  0.28409731486251916
option price:  6.949343041734153
